# Open AI mudelite peenhäälestus

See märkmik põhineb Open AI [Peenhäälestuse](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) dokumentatsioonis antud praegustel juhistel.

Peenhäälestus parandab põhjalike mudelite jõudlust teie rakenduse jaoks, treenides seda täiendavate andmete ja konkreetse kasutusjuhtumi või stsenaariumiga seotud kontekstiga uuesti. Pange tähele, et promptide inseneritehnikad nagu _few shot learning_ ja _retrieval augmented generation_ võimaldavad teil täiendada vaikimisi prompti asjakohaste andmetega kvaliteedi parandamiseks. Kuid need lähenemised on piiratud sihtmudeli maksimaalse tokeni akna suurusega.

Peenhäälestusega treenime mudelit tegelikult uuesti vajalike andmetega (mis võimaldab kasutada palju rohkem näiteid kui mahub maksimaalsesse tokeni aknasse) - ja juurutame _kohandatud_ mudeli versiooni, millel ei ole enam vaja pakkuda näiteid järeldusajal. See mitte ainult ei paranda meie prompti kujunduse tõhusust (meil on rohkem paindlikkust tokeni akna kasutamisel muude asjade jaoks), vaid võib potentsiaalselt parandada ka meie kulusid (vähendades tokenite arvu, mida peame mudelile järeldusajal saatma).

Peenhäälestusel on 4 sammu:
1. Valmistage ette treeningandmed ja laadige need üles.
1. Käivitage treeningülesanne, et saada peenhäälestatud mudel.
1. Hinnake peenhäälestatud mudelit ja korrake kvaliteedi parandamiseks.
1. Juurutage peenhäälestatud mudel järeldamiseks, kui olete rahul.

Pange tähele, et mitte kõik põhjalikud mudelid ei toeta peenhäälestust - [vaadake OpenAI dokumentatsiooni](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) viimase teabe saamiseks. Samuti saate peenhäälestada varem peenhäälestatud mudelit. Selles juhendis kasutame peenhäälestuse sihtmudelina `gpt-35-turbo`. 

---


### Samm 1.1: Valmista oma andmestik ette

Loome vestlusroboti, mis aitab sul mõista keemiliste elementide perioodilisustabelit, vastates küsimustele elementide kohta limeriku vormis. Selles lihtsas juhendis loome andmestiku, et mudelit treenida mõne näidiskvastusega, mis näitab andmete oodatud vormingut. Tõsises kasutusjuhtumis peaksid looma palju suurema hulga näidete andmestiku. Võid ka kasutada avatud andmestikku (oma rakenduse valdkonna jaoks), kui see on olemas, ja vormindada see ümber peenhäälestuseks sobivaks.

Kuna keskendume `gpt-35-turbo` mudelile ja otsime ühekäigulist vastust (vestluse lõpetamine), saame luua näited kasutades [seda soovitatud vormingut](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst), mis vastab OpenAI vestluse lõpetamise nõuetele. Kui ootad mitmekäigulist vestlust, kasutaksid [mitmekäigulise näite vormingut](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst), mis sisaldab `weight` parameetrit, et näidata, milliseid sõnumeid peenhäälestamisel kasutada (või mitte).

Me kasutame siin juhendis lihtsamat ühekäigulist vormingut. Andmed on [jsonl vormingus](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst), kus iga rida sisaldab ühte kirjet, mis on esitatud JSON-formaadis objektina. Allolev näide näitab 2 kirjet - vaata [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) täielikku näidiskomplekti (10 näidet), mida kasutame peenhäälestuse juhendis. **Märkus:** Iga kirje _peab_ olema määratletud ühel real (mitte jagatud mitmele reale nagu tavapärases vormindatud JSON-failis).

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

Tõsises kasutusjuhtumis vajad palju suuremat näidete komplekti hea tulemuse saavutamiseks - kompromiss seisneb vastuste kvaliteedi ja peenhäälestuse aja/kulude vahel. Me kasutame väikest komplekti, et peenhäälestuse protsessi kiiresti lõpetada ja seda illustreerida. Vaata [seda OpenAI kokaraamatu näidet](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) keerukama peenhäälestuse juhendi jaoks.


---

### Samm 1.2 Laadige oma andmestik üles

Laadige andmed üles kasutades Files API-d [nagu siin kirjeldatud](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Pange tähele, et selle koodi käivitamiseks peate esmalt tegema järgmised sammud:
 - Paigaldanud `openai` Python paketi (veenduge, et kasutate versiooni >=0.28.0 uusimate funktsioonide jaoks)
 - Määranud keskkonnamuutuja `OPENAI_API_KEY` oma OpenAI API võtmele
Lisateabe saamiseks vaadake kursuse jaoks pakutud [Seadistamise juhendit](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst).

Nüüd käivitage kood, et luua fail üleslaadimiseks teie kohalikust JSONL failist.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


---

### Samm 2.1: Loo peenhäälestamise töö SDK-ga


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


---

### Samm 2.2: Kontrolli töö olekut

Siin on mõned asjad, mida saad teha `client.fine_tuning.jobs` API-ga:
- `client.fine_tuning.jobs.list(limit=<n>)` - Loetle viimased n peenhäälestustööd
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Hangi konkreetse peenhäälestustöö detailid
- `client.fine_tuning.jobs.cancel(<job_id>)` - Tühista peenhäälestustöö
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - Loetle kuni n sündmust tööst
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Protsessi esimene samm on _treeningfaili valideerimine_, et veenduda andmete õiges formaadis olemises.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


---

### Samm 2.3: Jälgi sündmusi, et edusamme monitorida


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Samm 2.4: Vaata olekut OpenAI juhtpaneelil


Saate olekut vaadata ka, külastades OpenAI veebisaiti ja uurides platvormi _Fine-tuning_ jaotist. See näitab teile praeguse töö olekut ning võimaldab jälgida varasemate tööde täitmise ajalugu. Selles ekraanipildis näete, et eelmine täitmine ebaõnnestus ja teine kordamine õnnestus. Konteksti mõttes juhtus see siis, kui esimesel korral kasutati JSON-faili valesti vormindatud kirjetega – pärast parandamist lõpetas teine kordamine edukalt ja mudel sai kasutamiseks kättesaadavaks.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.et.png)


Saate olekuteateid ja mõõdikuid vaadata ka visuaalses juhtpaneelis allapoole kerides, nagu on näidatud:

| Messages | Metrics |
|:---|:---|
| ![Messages](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.et.png) |  ![Metrics](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.et.png)|


---

### Samm 3.1: Hangi ID ja testi peenhäälestatud mudelit koodis


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


---

### Samm 3.2: Laadi ja testi peenhäälestatud mudelit mänguväljakul

Nüüd saate peenhäälestatud mudelit testida kahel viisil. Esiteks võite külastada mänguväljakut ja kasutada mudelite rippmenüüd, et valida oma äsja peenhäälestatud mudel valikute seast. Teine võimalus on kasutada peenhäälestuse paneelil kuvatavat "Playground" valikut (vt ülalolevat ekraanipilti), mis avab järgmise _võrdleva_ vaate, kus näidatakse alusmudeli ja peenhäälestatud mudeli versioone kõrvuti kiireks hindamiseks.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.et.png)

Täida lihtsalt süsteemi kontekst, mida kasutati sinu treeningandmetes, ja esita oma testküsimus. Märkad, et mõlemad pooled uuendatakse identse konteksti ja küsimusega. Käivita võrdlus ja näed nende väljundite erinevust. _Pane tähele, kuidas peenhäälestatud mudel esitab vastuse sinu näidetes antud vormingus, samas kui alusmudel järgib lihtsalt süsteemi prompti_.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.et.png)

Märgid, et võrdlus näitab ka iga mudeli tokenite arvu ja inferentsi jaoks kulunud aega. **See konkreetne näide on lihtsustatud, et näidata protsessi, kuid ei peegelda tegelikku andmekogumit ega stsenaariumi**. Võid märgata, et mõlemad näited näitavad sama arvu tokeneid (süsteemi kontekst ja kasutaja prompt on identsed), kuid peenhäälestatud mudel võtab inferentsiks rohkem aega (kohandatud mudel).

Tegelikes olukordades ei kasuta sa sellist mänguasjanäidet, vaid peenhäälestad reaalse andmestiku vastu (nt tootekataloog klienditeeninduseks), kus vastuse kvaliteet on palju ilmsem. _Selles_ kontekstis nõuab võrreldava vastuse kvaliteedi saavutamine alusmudeliga rohkem kohandatud promptide loomist, mis suurendab tokenite kasutust ja potentsiaalselt ka inferentsi töötlemise aega. _Selle proovimiseks vaata OpenAI Cookbooki peenhäälestuse näiteid._

---


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Vastutusest loobumine**:
See dokument on tõlgitud kasutades tehisintellektil põhinevat tõlketeenust [Co-op Translator](https://github.com/Azure/co-op-translator). Kuigi püüame tagada täpsust, palun arvestage, et automaatsed tõlked võivad sisaldada vigu või ebatäpsusi. Originaaldokument selle emakeeles tuleks pidada autoriteetseks allikaks. Olulise teabe puhul soovitatakse kasutada professionaalset inimtõlget. Me ei vastuta selle tõlke kasutamisest tulenevate arusaamatuste või valesti mõistmiste eest.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
